In [1]:
from datetime import datetime
def timecode_to_seconds(timecode):
    time_format = "%H:%M:%S.%f"  # Bao gồm cả phần mili giây
    dt = datetime.strptime(timecode, time_format)
    return dt.hour * 3600 + dt.minute * 60 + dt.second + dt.microsecond / 1_000_000

In [2]:
import pandas as pd
from datetime import datetime

# Define the conversion function
def timecode_to_seconds(timecode):
    time_format = "%H:%M:%S.%f"  # Including milliseconds
    dt = datetime.strptime(timecode, time_format)
    return dt.hour * 3600 + dt.minute * 60 + dt.second + dt.microsecond / 1_000_000

# Load the data from the CSV file
data = pd.read_csv('scene_durations.csv')

# Convert 'Start Time' and 'End Time' columns to seconds
data['Start Time'] = data['Start Time'].apply(timecode_to_seconds)
data['End Time'] = data['End Time'].apply(timecode_to_seconds)

# Calculate 'Real duration'
data['Real duration'] = data['End Time'] - data['Start Time']

# Display the first 5 rows
data


,Scene,Start Time,End Time,Duration (seconds),Real duration
0,1,0.000,1.168,2,1.168
1,2,1.168,4.471,4,3.303
2,3,4.471,5.906,2,1.435
3,4,5.906,7.474,2,1.568
4,5,7.474,8.642,2,1.168
5,6,8.642,10.277,2,1.635
6,7,10.277,11.645,2,1.368
7,8,11.645,14.882,4,3.237
8,9,14.882,38.872,24,23.990
9,10,38.872,57.891,20,19.019


In [3]:
from tqdm import tqdm
import pandas as pd
import math

# Ensure column names are correct
empty_df = pd.DataFrame(columns=['Scene', 'Start Time', 'End Time', 'Duration (seconds)'])

# Iterate over the DataFrame in steps of 2
for i in tqdm(range(0, len(data), 2)):
    # Check if the 'Real duration' is less than 2 for the current or next row
    if data.iloc[i]['Real duration'] < 1 or (i + 1 < len(data) and data.iloc[i + 1]['Real duration'] < 1):
        # Create a new row with merged information
        new_row = pd.DataFrame([{
            'Scene': data.iloc[i]['Scene'],
            'Start Time': data.iloc[i]['Start Time'],
            'End Time': data.iloc[i + 1]['End Time'] if i + 1 < len(data) else data.iloc[i]['End Time'],
            'Duration (seconds)': math.ceil(data.iloc[i]['Real duration'] + (data.iloc[i + 1]['Real duration'] if i + 1 < len(data) else 0))
        }])
        # Concatenate the new row to empty_df
        empty_df = pd.concat([empty_df, new_row], ignore_index=True)
    else:
        # Add individual rows if no merging is needed
        row_1 = data.iloc[[i]]
        empty_df = pd.concat([empty_df, row_1], ignore_index=True)

        if i + 1 < len(data):
            row_2 = data.iloc[[i + 1]]
            empty_df = pd.concat([empty_df, row_2], ignore_index=True)

empty_df = empty_df.drop(columns='Real duration')
empty_df['Scene'] = empty_df.index + 1


  0%|          | 0/28 [00:00<?, ?it/s]/tmp/ipykernel_115019/333652227.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  empty_df = pd.concat([empty_df, row_1], ignore_index=True)
100%|██████████| 28/28 [00:00<00:00, 667.63it/s]


In [7]:
empty_df.to_csv('scene_durations_v1.csv', index = False)